In [83]:
import pandas as pd 
import numpy as np
import requests
import urllib

The purpose of this Notebook, is to add a column to the citibike table with the geoid corresponding to the census tract which the citibike station is located in. 

In [84]:
#This is a table containing all of the stations owned by citibike in NYC. 
stations = pd.read_csv('stations.csv')
stations = stations[stations['lat'] != 45.506264191001186] 
#remove this station since it is located in canada not NYC.
#45.506264191001186, -73.56890559196472

In [85]:
#This gets the (lat,lon) of all of the citibike stations in NYC 
stations_lat = list(stations['lat'])
stations_lon = list(stations['lon'])
stations_lat_lon = [(stations_lat[i],stations_lon[i]) for i in range(len(stations_lat))]

In [88]:
#This uses a freely available goverment API that allows us to get the geoid based on the (lat,lon) of each citibike sttaion.
station_geoid = []
for lat, lon in stations_lat_lon: 
    #Encode parameters 
    params = urllib.parse.urlencode({'latitude': lat, 'longitude':lon, 'format':'json'})
    #Contruct request URL
    url = 'https://geo.fcc.gov/api/census/block/find?' + params

    #Get response from API
    response = requests.get(url)

    #Parse json in response
    data = response.json()

    #Print FIPS code
    returned_FIPS = (data['Block']['FIPS'])
    geoid = returned_FIPS[:11] 
    station_geoid.append(geoid)

In [89]:
stations_geoid = [int(geoid) for geoid in station_geoid]

In [91]:
#add the geoid column to the stations table
stations['geoid'] = stations_geoid

In [93]:
stations.to_csv('citibike_stations_geoid',index=False)